# Googele PEGASUS Abstractive Text Summarization 
#### Pre-training with Extracted Gap-sentenes for Abstractive Summaries

### Transformer model
A FFNN s2s model with input sequence to output sequence set of encoders and decoders. However each layer of the encoder and deecoder has a feed forward layer 

### Masked Language Model
Randomly masks 15% of tokens and uses the output to predict the masked word using FFNN + Softmax activation, trained in an unsupervised way 

### Scroing Metrics
#### ROGUE F1 - Recall-Orientated Understudy for Gisting Evaluation
Rogue F1 refers to the unigram crossover between the system generated summary and the reference summary (Human derived)
* recall = number of overlapping words / total words in reference summary
* precision = number of overlapping words / total words in system summary

### Gap Sentence Generation 

* Select and mask whole sents, concat gap sents into pseudo summary
* Corresponding position of gap sent is replacd by MASK
* Gap sentence ratio considers the number of gap sents to total doc sents
* Select sents that appear to be most important/principal sents withing doc
* Result has empirically demostrated benifits and anticipates the downstream task

### Gap Sentence Generation Strategies
Three primary strategies for selecting m gap sents without replacement from document D = {Xi}n, comprised of n sentences
* Random - uniformly select m sentences at random
* Lead select the first m sentences
* Principal select the top-m scored sents according to importance. As a proxy for importance we compute ROUGE1-F1 between sentences and the rest of the document, Si = rouge(Xi,D\{Xi}, Vi

In [1]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Pegasis XSum

In [32]:
ml = ["""Students are often tasked with reading a document and producing a summary (for example, a book report) to demonstrate both reading comprehension and writing ability. This abstractive text summarization is one of the most challenging tasks in natural language processing, involving understanding of long passages, information compression, and language generation. The dominant paradigm for training machine learning models to do this is sequence-to-sequence (seq2seq) learning, where a neural network learns to map input sequences to output sequences. While these seq2seq models were initially developed using recurrent neural networks, Transformer encoder-decoder models have recently become favored as they are more effective at modeling the dependencies present in the long sequences encountered in summarization.

Transformer models combined with self-supervised pre-training (e.g., BERT, GPT-2, RoBERTa, XLNet, ALBERT, T5, ELECTRA) have shown to be a powerful framework for producing general language learning, achieving state-of-the-art performance when fine-tuned on a wide array of language tasks. In prior work, the self-supervised objectives used in pre-training have been somewhat agnostic to the down-stream application in favor of generality; we wondered whether better performance could be achieved if the self-supervised objective more closely mirrored the final task.

In “PEGASUS: Pre-training with Extracted Gap-sentences for Abstractive Summarization” (to appear at the 2020 International Conference on Machine Learning), we designed a pre-training self-supervised objective (called gap-sentence generation) for Transformer encoder-decoder models to improve fine-tuning performance on abstractive summarization, achieving state-of-the-art results on 12 diverse summarization datasets. Supplementary to the paper, we are also releasing the training code and model checkpoints on GitHub.

A Self-Supervised Objective for Summarization
Our hypothesis is that the closer the pre-training self-supervised objective is to the final down-stream task, the better the fine-tuning performance. In PEGASUS pre-training, several whole sentences are removed from documents and the model is tasked with recovering them. An example input for pre-training is a document with missing sentences, while the output consists of the missing sentences concatenated together. This is an incredibly difficult task that may seem impossible, even for people, and we don’t expect the model to solve it perfectly. However, such a challenging task encourages the model to learn about language and general facts about the world, as well as how to distill information taken from throughout a document in order to generate output that closely resembles the fine-tuning summarization task. The advantage of this self-supervision is that you can create as many examples as there are documents, without any human annotation, which is often the bottleneck in purely supervised systems.

A self-supervised example for PEGASUS during pre-training. The model is trained to output all the masked sentences.
We found that choosing “important” sentences to mask worked best, making the output of self-supervised examples even more similar to a summary. We automatically identified these sentences by finding those that were most similar to the rest of the document according to a metric called ROUGE. ROUGE computes the similarity of two texts by computing n-gram overlaps using a score from 0 to 100 (ROUGE-1, ROUGE-2, and ROUGE-L are three common variants).

Similar to other recent methods, such as T5, we pre-trained our model on a very large corpus of web-crawled documents, then we fine-tuned the model on 12 public down-stream abstractive summarization datasets, resulting in new state-of-the-art results as measured by automatic metrics, while using only 5% of the number of parameters of T5. The datasets were chosen to be diverse, including news articles, scientific papers, patents, short stories, e-mails, legal documents, and how-to directions, showing that the model framework is adaptive to a wide-variety of topics."""]

In [26]:
model_name = 'google/pegasus-xsum'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [27]:
batch = tokenizer.prepare_seq2seq_batch(ml, truncation=True, padding='longest', return_tensors='pt').to(torch_device)

translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
print(tgt_text)

['We have developed a new approach for training machine learning models to perform abstractive text summarization.']


# Pegasus Large

In [36]:
model_name = 'google/pegasus-large'
torch_device = 'cude' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [37]:
batch = tokenizer.prepare_seq2seq_batch(ml, truncation=True, padding= 'longest', return_tensors='pt').to(torch_device)

translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
print(tgt_text)

['Transformer models combined with self-supervised pre-training (e.g., BERT, GPT-2, RoBERTa, XLNet, ALBERT, T5, ELECTRA) have shown to be a powerful framework for producing general language learning, achieving state-of-the-art performance when fine-tuned on a wide array of language tasks. In “PEGASUS: Pre-training with Extracted Gap-sentences for Abstractive Summarization” (to appear at the 2020 International Conference on Machine Learning), we designed a pre-training self-supervised objective (called gap-sentence generation) for Transformer encoder-decoder models to improve fine-tuning performance on abstractive summarization, achieving state-of-the-art results on 12 diverse summarization datasets.']


# Pegasus Reddit Tifu

In [30]:
model_name = 'google/pegasus-reddit_tifu'
torch_device = 'cuda'if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [33]:
batch = tokenizer.prepare_seq2seq_batch(ml, truncation=True, padding='longest', return_tensors='pt').to(torch_device)

translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens = True)
print(tgt_text)

['we designed a pre-training self-supervised objective (called gap-sentence generation) for Transformer encoder-decoder models to improve fine-tuning performance on abstractive summarization, achieving state-of-the-art results on 12 diverse summarization datasets.']
